# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [1]:
import tensorflow as tf

In [2]:
from utils import get_dataset

In [3]:
import os
tfrecex = "segment-141184560845819621_10582_560_10602_560_with_camera_labels.tfrecord"
waymo_path = "{cwd}/data/waymo/{tfrec}".format(cwd = os.getcwd(), tfrec = tfrecex)
# print(os.listdir(waymo_path))

In [5]:
dataset = get_dataset(waymo_path)
print(type(dataset))


INFO:tensorflow:Reading unweighted datasets: ['/app/project/nd013-c1-vision-starter/data/waymo/segment-141184560845819621_10582_560_10602_560_with_camera_labels.tfrecord']
INFO:tensorflow:Reading record datasets for input file: ['/app/project/nd013-c1-vision-starter/data/waymo/segment-141184560845819621_10582_560_10602_560_with_camera_labels.tfrecord']
INFO:tensorflow:Number of filenames to read: 1
<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [7]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    """
    for sample in batch:
        
    

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [8]:
dataset.shuffle(100)
display_instances(dataset.take(10))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa9 in position 40: invalid start byte

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...